In [1]:
import os
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)

# Adjust this path if needed
COMPETITION_PATH = "."


def load_competition_datasets(data_dir, sample_frac=None, random_state=None):
    """
    Load train and test datasets, optionally sample a fraction of the training set,
    concatenate, and reset index.
    """
    print("Loading competition datasets from:", data_dir)
    train_file = os.path.join(data_dir, "/kaggle/input/dataset/train_data.txt")
    test_file = os.path.join(data_dir, "/kaggle/input/dataset/test_data.txt")

    # Load training data and optionally subsample
    train_df = pd.read_csv(train_file, sep="\t", low_memory=False)
    if sample_frac is not None:
        train_df = train_df.sample(frac=sample_frac, random_state=random_state)

    # Load test data
    test_df = pd.read_csv(test_file, sep="\t", low_memory=False)

    # Concatenate and reset index
    combined = pd.concat([train_df, test_df], ignore_index=True)
    print(f"  → Concatenated DataFrame: {combined.shape[0]} rows")
    return combined


def cast_column_types(df):
    """
    Cast columns to efficient dtypes and parse datetime fields.
    """
    print("Casting column types and parsing datetime fields...")
    dtype_map = {
        "platform": "category",
        "conn_country": "category",
        "ip_addr": "category",
        "master_metadata_track_name": "category",
        "master_metadata_album_artist_name": "category",
        "master_metadata_album_album_name": "category",
        "reason_end": "category",
        "username": "category",
        "spotify_track_uri": "string",
        "episode_name": "string",
        "episode_show_name": "string",
        "spotify_episode_uri": "string",
        "audiobook_title": "string",
        "audiobook_uri": "string",
        "audiobook_chapter_uri": "string",
        "audiobook_chapter_title": "string",
        "shuffle": bool,
        "offline": bool,
        "incognito_mode": bool,
        "obs_id": int,
    }

    df["ts"] = pd.to_datetime(df["ts"], utc=True)
    df["offline_timestamp"] = pd.to_datetime(
        df["offline_timestamp"], unit="s", errors="coerce", utc=True
    )
    df = df.astype(dtype_map)
    print("  → Column types cast successfully.")
    return df


def split_train_test(X, y, test_mask):
    """
    Split features and labels into train/test based on mask.
    """
    print("Splitting data into train/test sets...")
    
    X_train = X
    X_test = X
    y_train = y
    y_test = y

    print(f"  → Training set: {X_train.shape[0]} rows")
    print(f"  → Test set:     {X_test.shape[0]} rows")
    return X_train, X_test, y_train, y_test


def train_classifier(X_train, y_train, params=None):
    """
    Train a Classifier 
    """
    print("Training model...")
    
    default_params = {
        "n_estimators": 100,
        "max_depth": None,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "max_features": "sqrt",
        "n_jobs": -1,
        "random_state": 42,
        "bootstrap": True,
    }
    rf_params = default_params.copy()
    if params:
        rf_params.update(params)

    model = RandomForestClassifier(**rf_params)

    print("  → Fitting RandomForestClassifier...")
    model.fit(X_train, y_train)
    print("  → Model training complete.")
    return model





def main():
    print("=== Starting pipeline ===")

    # Load and preprocess data
    df = load_competition_datasets(
        COMPETITION_PATH, sample_frac=0.2, random_state=1234
    )
    df = cast_column_types(df)

    # Generate user order column
    df = df.sort_values(["username", "ts"])
    df["user_order"] = df.groupby("username", observed=True).cumcount() + 1
    df = df.sort_values(["obs_id"])

    
    # Create target and test mask

    print("Creating 'target' and 'is_test' columns...")
    df["target"] = (df["reason_end"] == "fwdbtn").astype(int)
    df["is_test"] = df["reason_end"].isna()
    df.drop(columns=["reason_end"], inplace=True)
    print("  → 'target' and 'is_test' created, dropped 'reason_end' column.")

    to_keep = [
        "obs_id",
        "target",
        "is_test",
        "user_order"
    ] #MODIFICAR
    df = df[to_keep]

   # Build feature matrix and get feature names
    y = df["target"].to_numpy()
    X = df.drop(columns=["target"])
    feature_names = X.columns
    test_mask = df["is_test"].to_numpy()

    # Split data
    X_train, X_test, y_train, _ = split_train_test(X, y, test_mask)

    # Train model
    model = train_classifier(X_train, y_train)

    # Display top 20 feature importances
    print("Extracting and sorting feature importances...")
    importances = model.feature_importances_
    imp_series = pd.Series(importances, index=feature_names)
    imp_sorted = imp_series.sort_values(ascending=False)
    print("\nTop 20 feature importances:")
    print(imp_sorted.head(20))

    # Predict on test set
    print("Generating predictions for test set...")
    test_obs_ids = X_test["obs_id"]
    preds_proba = model.predict_proba(X_test)[:, 1]
    preds_df = pd.DataFrame({"obs_id": test_obs_ids, "pred_proba": preds_proba})
    preds_df.to_csv("modelo_benchmark.csv", index=False, sep=",")
    print(f"  → Predictions written to 'modelo_benchmark.csv'")

    print("=== Pipeline complete ===")


if __name__ == "__main__":
    main()

=== Starting pipeline ===
Loading competition datasets from: .
  → Concatenated DataFrame: 233839 rows
Casting column types and parsing datetime fields...
  → Column types cast successfully.
Creating 'target' and 'is_test' columns...
  → 'target' and 'is_test' created, dropped 'reason_end' column.
Splitting data into train/test sets...
  → Training set: 233839 rows
  → Test set:     233839 rows
Training model...
  → Fitting RandomForestClassifier...
  → Model training complete.
Extracting and sorting feature importances...

Top 20 feature importances:
obs_id        0.514312
user_order    0.461868
is_test       0.023820
dtype: float64
Generating predictions for test set...
  → Predictions written to 'modelo_benchmark.csv'
=== Pipeline complete ===


# **XGBoost**



In [2]:
import pandas as pd # Para cargar los datos y hacer OHE.
import numpy as np  # Para lidiar con NaNs.
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import confusion_matrix



random_state = 42
np.random.seed(random_state)

In [3]:
train_data = pd.read_csv("/kaggle/input/dataset/train_data.txt", sep="\t", low_memory=False, on_bad_lines='skip')
test_data = pd.read_csv("/kaggle/input/dataset/test_data.txt", sep="\t", low_memory=False, on_bad_lines='skip')
print(train_data.shape)
print(test_data.shape)

(911344, 22)
(51570, 21)


In [4]:
train_data.head()


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ts,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_end,shuffle,offline,offline_timestamp,incognito_mode,username,obs_id
0,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,The Eater Of Dreams,Nine Inch Nails,Hesitation Marks,spotify:track:1IPdwxRUbuNZiRpFN49RQC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwdbtn,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,1
1,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Copy Of A,Nine Inch Nails,Hesitation Marks,spotify:track:4BFKCEp4gwG3QHNlYodLMy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,2
2,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,El extranjero,Bunbury,Singles,spotify:track:7ghPo5JVqcmwZQSpm6uyGG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trackdone,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,3
3,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Luna,Bunbury,Singles,spotify:track:4g0Jk6sWxdu3FwwGDtoQPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwdbtn,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,4
4,2013-10-29 18:40:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Feliz año,Bunbury,Singles,spotify:track:0GZzkyz7MN15llCyCBgPWh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwdbtn,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,5


In [5]:
train_data.describe()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,obs_id
count,0.0,0.0,0.0,0.0,911344.000000
mean,NaN,NaN,NaN,NaN,455672.500000
std,NaN,NaN,NaN,NaN,263082.496201
min,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,227836.750000
50%,NaN,NaN,NaN,NaN,455672.500000
75%,NaN,NaN,NaN,NaN,683508.250000
max,NaN,NaN,NaN,NaN,911344.000000


In [6]:
train_data.describe(include = 'object')

,ts,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_end,offline_timestamp,username
count,911344,911344,911344,911344,907211,907211,907211,907211,4133,4133,4133,911344,200888,911344
unique,113343,570,43,18444,95837,18543,41445,129731,1846,349,1861,15,24346,10
top,2019-02-07 21:40:00+00:00,ios,AR,d28b0c9777b108ac7f5ab057c4a7da93a8e2c2bf0c941a...,I Feel It Coming,Patricio Rey y sus Redonditos de Ricota,Nancy & Lee,spotify:track:1jJci4qxiYcOHhQR247rEU,La Zorra y el Conejo,Las Nuevas Historias de CoComelon,spotify:episode:42geAoXFSiFulSIRpfy03e,trackdone,1970-01-01 00:00:00+00:00,f15bafb0dc3331899681fffc6fbf48c10576b10e4a9044...
freq,424,79881,816259,16465,746,10912,3386,585,41,589,41,521220,2036,222644


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911344 entries, 0 to 911343
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 911344 non-null  object 
 1   platform                           911344 non-null  object 
 2   conn_country                       911344 non-null  object 
 3   ip_addr                            911344 non-null  object 
 4   master_metadata_track_name         907211 non-null  object 
 5   master_metadata_album_artist_name  907211 non-null  object 
 6   master_metadata_album_album_name   907211 non-null  object 
 7   spotify_track_uri                  907211 non-null  object 
 8   episode_name                       4133 non-null    object 
 9   episode_show_name                  4133 non-null    object 
 10  spotify_episode_uri                4133 non-null    object 
 11  audiobook_title                    0 no

In [8]:
# Importante: ¡sólo poner las categóricas y excluir la variable a predecir!
pd_ohe = pd.get_dummies(train_data,
                        columns = ['platform', 'conn_country'],
                        sparse = True,    # Devolver una matriz rala.
                        dummy_na = False, # No agregar columna para NaNs.
                        dtype = int       # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )
pd_ohe

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,ts,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_end,shuffle,offline,offline_timestamp,incognito_mode,username,obs_id,"platform_Android OS 10 API 29 (motorola, moto g(9) plus)","platform_Android OS 11 API 30 (motorola, moto g(9) plus)","platform_Android OS 11 API 30 (motorola, moto g200 5G)","platform_Android OS 11 API 30 (motorola, moto g71 5G)","platform_Android OS 12 API 31 (motorola, moto g200 5G)","platform_Android OS 12 API 31 (motorola, moto g71 5G)","platform_Android OS 12 API 31 (samsung, SM-G990E)","platform_Android OS 4.1.2 API 16 (samsung, GT-I8260L)","platform_Android OS 4.1.2 API 16 (samsung, GT-S7390L)","platform_Android OS 4.4.2 API 19 (samsung, SM-G355M)","platform_Android OS 4.4.4 API 19 (motorola, XT1045)","platform_Android OS 5.0.2 API 21 (motorola, MotoE2(4G-LTE))","platform_Android OS 5.1 API 22 (BLU, BLU LIFE ONE X)","platform_Android OS 5.1 API 22 (alps, i5)","platform_Android OS 5.1 API 22 (motorola, XT1032)","platform_Android OS 5.1 API 22 (motorola, XT1045)","platform_Android OS 5.1.1 API 22 (LGE, LG-H735)","platform_Android OS 5.1.1 API 22 (motorola, MotoG3)","platform_Android OS 6.0 API 23 (LGE, LG-H735)","platform_Android OS 6.0 API 23 (motorola, MotoG3)","platform_Android OS 6.0.1 API 23 (samsung, SM-G532M)","platform_Android OS 7.0 API 24 (LGE, LG-H860)","platform_Android OS 7.0 API 24 (motorola, Moto C Plus)","platform_Android OS 7.0 API 24 (motorola, Moto G (5) Plus)","platform_Android OS 7.0 API 24 (motorola, XT1575)","platform_Android OS 7.0 API 24 (samsung, SM-G925I)","platform_Android OS 8.1.0 API 27 (HUAWEI, JKM-LX2)","platform_Android OS 8.1.0 API 27 (motorola, Moto G (5) Plus)","platform_Android OS 9 API 28 (HUAWEI, JKM-LX2)","platform_Android-tablet OS 5.0.2 API 21 (samsung, SM-P550)",platform_Linux Ubuntu Core 18 (snap package) [x86-64 0],platform_Linux [x86-64 0],platform_OS X 10.11.6 [x86 8],platform_OS X 10.12.2 [x86 4],platform_OS X 10.12.3 [x86 8],platform_OS X 10.12.4 [x86 8],platform_OS X 10.12.5 [x86 8],platform_OS X 10.12.6 [x86 8],platform_OS X 10.13.1 [x86 8],platform_OS X 10.13.2 [x86 8],platform_OS X 10.13.3 [x86 8],platform_OS X 10.13.4 [x86 8],platform_OS X 10.13.5 [x86 8],platform_OS X 10.13.6 [x86 8],platform_OS X 10.14.0 [x86 8],platform_OS X 10.14.3 [x86 8],platform_OS X 10.14.5 [x86 8],platform_OS X 10.14.6 [x86 8],platform_OS X 10.15.1 [x86 8],platform_OS X 10.15.2 [x86 8],platform_OS X 10.15.4 [x86 8],platform_OS X 10.15.5 [x86 8],platform_OS X 10.15.6 [x86 8],platform_OS X 10.15.7 [x86 8],platform_OS X 10.16.0 [x86 4],platform_OS X 10.16.0 [x86 8],platform_OS X 11.2.1 [x86 8],platform_OS X 11.2.3 [x86 4],platform_OS X 11.2.3 [x86 8],platform_OS X 11.3.0 [x86 8],platform_OS X 11.4.0 [x86 8],platform_OS X 11.5.1 [x86 8],platform_OS X 11.5.2 [x86 8],platform_OS X 11.6.0 [arm 2],platform_OS X 11.6.0 [x86 4],platform_OS X 11.6.0 [x86 8],platform_OS X 11.6.2 [x86 8],platform_OS X 11.6.5 [x86 8],platform_OS X 11.6.8 [x86 8],platform_OS X 12.0.1 [arm 2],platform_OS X 12.4.0 [arm 2],platform_OS X 12.5.1 [arm 2],platform_OS X 12.6.0 [arm 2],platform_Partner apple_tv apple;apple_tv;33ddb95064d1479ab37179579af23b77;;tpapi,platform_Partner applewatch_sdk Apple;Watch3.4;a6def90dca074656984e6e6968bd5866;7.3.3;,platform_Partner google cast_tv;BeyondTV2;;5.15.0--1.56.500000,platform_Partner google cast_tv;Chromecast;;,platform_Partner google cast_tv;Chromecast;;1.22.78337,platform_Partner google cast_tv;Chromecast;;1.24.88007,platform_Partner google cast_tv;Chromecast;;2.0.0-357-22b28ee--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-463-fe838b1--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-482-277fa0d--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-511-58a5275--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-511-

In [9]:
len(pd_ohe.columns)

633

In [10]:
pd_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911344 entries, 0 to 911343
Columns: 633 entries, ts to conn_country_ZZ
dtypes: Sparse[int64, 0](613), bool(3), float64(4), int64(1), object(12)
memory usage: 141.7+ MB


### **Preparar conjuntos de entrenamiento, validación (*hold-out*) y evaluación**

In [11]:
# fwdbtn => 1 (skip), otro => 0
pd_ohe['y'] = (train_data['reason_end'] == 'fwdbtn').astype(int)

y = pd_ohe[['y']].copy()  # mantenemos tu patrón
y


,y
0,1
1,0
2,0
3,1
4,1
...,...
911339,0
911340,0
911341,0
911342,0


In [12]:
#saco el mapeo yes/no por 0/1
y['y'] = y['y'].astype(int)


In [13]:
y['y'].unique()

array([1, 0])

In [14]:
#CAMBIOOOO
X = pd_ohe.drop('y', axis = 1)


X = X.select_dtypes(include=['number','bool'])
try:
    X = X.sparse.to_dense()
except AttributeError:
    pass
X = X.astype('float32')

print("X shape:", X.shape)
X.head(2)


X shape: (911344, 621)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,shuffle,offline,incognito_mode,obs_id,"platform_Android OS 10 API 29 (motorola, moto g(9) plus)","platform_Android OS 11 API 30 (motorola, moto g(9) plus)","platform_Android OS 11 API 30 (motorola, moto g200 5G)","platform_Android OS 11 API 30 (motorola, moto g71 5G)","platform_Android OS 12 API 31 (motorola, moto g200 5G)","platform_Android OS 12 API 31 (motorola, moto g71 5G)","platform_Android OS 12 API 31 (samsung, SM-G990E)","platform_Android OS 4.1.2 API 16 (samsung, GT-I8260L)","platform_Android OS 4.1.2 API 16 (samsung, GT-S7390L)","platform_Android OS 4.4.2 API 19 (samsung, SM-G355M)","platform_Android OS 4.4.4 API 19 (motorola, XT1045)","platform_Android OS 5.0.2 API 21 (motorola, MotoE2(4G-LTE))","platform_Android OS 5.1 API 22 (BLU, BLU LIFE ONE X)","platform_Android OS 5.1 API 22 (alps, i5)","platform_Android OS 5.1 API 22 (motorola, XT1032)","platform_Android OS 5.1 API 22 (motorola, XT1045)","platform_Android OS 5.1.1 API 22 (LGE, LG-H735)","platform_Android OS 5.1.1 API 22 (motorola, MotoG3)","platform_Android OS 6.0 API 23 (LGE, LG-H735)","platform_Android OS 6.0 API 23 (motorola, MotoG3)","platform_Android OS 6.0.1 API 23 (samsung, SM-G532M)","platform_Android OS 7.0 API 24 (LGE, LG-H860)","platform_Android OS 7.0 API 24 (motorola, Moto C Plus)","platform_Android OS 7.0 API 24 (motorola, Moto G (5) Plus)","platform_Android OS 7.0 API 24 (motorola, XT1575)","platform_Android OS 7.0 API 24 (samsung, SM-G925I)","platform_Android OS 8.1.0 API 27 (HUAWEI, JKM-LX2)","platform_Android OS 8.1.0 API 27 (motorola, Moto G (5) Plus)","platform_Android OS 9 API 28 (HUAWEI, JKM-LX2)","platform_Android-tablet OS 5.0.2 API 21 (samsung, SM-P550)",platform_Linux Ubuntu Core 18 (snap package) [x86-64 0],platform_Linux [x86-64 0],platform_OS X 10.11.6 [x86 8],platform_OS X 10.12.2 [x86 4],platform_OS X 10.12.3 [x86 8],platform_OS X 10.12.4 [x86 8],platform_OS X 10.12.5 [x86 8],platform_OS X 10.12.6 [x86 8],platform_OS X 10.13.1 [x86 8],platform_OS X 10.13.2 [x86 8],platform_OS X 10.13.3 [x86 8],platform_OS X 10.13.4 [x86 8],platform_OS X 10.13.5 [x86 8],platform_OS X 10.13.6 [x86 8],platform_OS X 10.14.0 [x86 8],platform_OS X 10.14.3 [x86 8],platform_OS X 10.14.5 [x86 8],platform_OS X 10.14.6 [x86 8],platform_OS X 10.15.1 [x86 8],platform_OS X 10.15.2 [x86 8],platform_OS X 10.15.4 [x86 8],platform_OS X 10.15.5 [x86 8],platform_OS X 10.15.6 [x86 8],platform_OS X 10.15.7 [x86 8],platform_OS X 10.16.0 [x86 4],platform_OS X 10.16.0 [x86 8],platform_OS X 11.2.1 [x86 8],platform_OS X 11.2.3 [x86 4],platform_OS X 11.2.3 [x86 8],platform_OS X 11.3.0 [x86 8],platform_OS X 11.4.0 [x86 8],platform_OS X 11.5.1 [x86 8],platform_OS X 11.5.2 [x86 8],platform_OS X 11.6.0 [arm 2],platform_OS X 11.6.0 [x86 4],platform_OS X 11.6.0 [x86 8],platform_OS X 11.6.2 [x86 8],platform_OS X 11.6.5 [x86 8],platform_OS X 11.6.8 [x86 8],platform_OS X 12.0.1 [arm 2],platform_OS X 12.4.0 [arm 2],platform_OS X 12.5.1 [arm 2],platform_OS X 12.6.0 [arm 2],platform_Partner apple_tv apple;apple_tv;33ddb95064d1479ab37179579af23b77;;tpapi,platform_Partner applewatch_sdk Apple;Watch3.4;a6def90dca074656984e6e6968bd5866;7.3.3;,platform_Partner google cast_tv;BeyondTV2;;5.15.0--1.56.500000,platform_Partner google cast_tv;Chromecast;;,platform_Partner google cast_tv;Chromecast;;1.22.78337,platform_Partner google cast_tv;Chromecast;;1.24.88007,platform_Partner google cast_tv;Chromecast;;2.0.0-357-22b28ee--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-463-fe838b1--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-482-277fa0d--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-511-58a5275--1.29.104827,platform_Partner google cast_tv;Chromecast;;2.0.0-511-58a5275--1.32.124602,platform_Partner google cast_tv;Chromecast;;2.0.0-551-fb8ab0b--1.32.124602,platform_Partner google cast_tv;Chromecast;;2.0.0-594-48a0591--1.35.137090,platform_Partner google cast_tv;Chromecas

In [15]:
#cambio y por y['y']
val_test_size = 0.3  # Proporción de la suma del test de validación y del de test.

X_train, X_tmp, Y_train, Y_tmp = train_test_split(X, y['y'],
                                train_size=0.7,
                                random_state=random_state,
                                stratify=y['y']
)


In [16]:
X_val, X_test, Y_val, Y_test = train_test_split(X_tmp, Y_tmp,
                                                train_size=0.5,
                                                random_state=random_state,
                                                stratify=Y_tmp
)


In [17]:
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')
print(f'Cantidad de datos de test: {len(X_test)}')

Cantidad de datos de train: 637940
Cantidad de datos de validación: 136702
Cantidad de datos de test: 136702


## XGBoost

In [18]:
#!pip install -U xgboost
import xgboost as xgb

In [19]:
# si usabas seed, cambialo por random_state
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            random_state=random_state,
                            eval_metric='auc',
                            tree_method='hist')  # opcional, acelera

In [20]:
# Sanear nombres de columnas para XGBoost (no acepta [, ], <)
def _sanitize_cols(df):
    return df.rename(columns=lambda c: str(c).replace('[','(').replace(']',')').replace('<','lt'))

X_train = _sanitize_cols(X_train)
X_val   = _sanitize_cols(X_val)

# si después vas a predecir en test/kaggle, acordate de hacer lo mismo:
# X_test = _sanitize_cols(X_test)    # cuando lo tengas armado


In [21]:
clf_xgb.fit(X_train, Y_train, verbose = True, eval_set = [(X_val, Y_val)])

[0]	validation_0-auc:0.70793
[1]	validation_0-auc:0.72283
[2]	validation_0-auc:0.72768
[3]	validation_0-auc:0.73412
[4]	validation_0-auc:0.74263
[5]	validation_0-auc:0.74684
[6]	validation_0-auc:0.75035
[7]	validation_0-auc:0.75457
[8]	validation_0-auc:0.75859
[9]	validation_0-auc:0.76162
[10]	validation_0-auc:0.76469
[11]	validation_0-auc:0.76655
[12]	validation_0-auc:0.76830
[13]	validation_0-auc:0.77074
[14]	validation_0-auc:0.77408
[15]	validation_0-auc:0.77614
[16]	validation_0-auc:0.77984
[17]	validation_0-auc:0.78108
[18]	validation_0-auc:0.78198
[19]	validation_0-auc:0.78352
[20]	validation_0-auc:0.78540
[21]	validation_0-auc:0.78729
[22]	validation_0-auc:0.78847
[23]	validation_0-auc:0.78952
[24]	validation_0-auc:0.79097
[25]	validation_0-auc:0.79177
[26]	validation_0-auc:0.79258
[27]	validation_0-auc:0.79298
[28]	validation_0-auc:0.79411
[29]	validation_0-auc:0.79570
[30]	validation_0-auc:0.79656
[31]	validation_0-auc:0.79693
[32]	validation_0-auc:0.79737
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

### **Búsqueda de hiperparámetros**

In [22]:
from scipy.stats import uniform
params = {'max_depth': list(range(1, 40)),
          'learning_rate': uniform(scale = 0.2),
          'gamma': uniform(scale = 2),
          'reg_lambda': uniform(scale = 5),        # Parámetro de regularización.
          'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
          'min_child_weight': uniform(scale = 5),
          'colsample_bytree': uniform(0.75, 0.25), # Entre 0.75 y 1.
          'n_estimators': list(range(1, 1000))
         }

In [23]:
import time, sklearn
from sklearn.model_selection import ParameterSampler
import xgboost as xgb

start = time.time()
best_score = 0.0
best_estimator = None
iterations = 100

# Para XGBoost >= 2.0:
base_kwargs = dict(
    objective='binary:logistic',
    eval_metric='auc',
    # fuerza GPU si hay CUDA
    device='cuda',                # <— clave
    random_state=random_state
)

# Para XGBoost < 2.0, en su defecto:
# base_kwargs.update(tree_method='gpu_hist', predictor='gpu_predictor')

for g in ParameterSampler(params, n_iter=iterations, random_state=random_state):
    clf_xgb = xgb.XGBClassifier(**base_kwargs, **g)
    clf_xgb.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], verbose=False)

    y_pred = clf_xgb.predict_proba(X_val)[:, 1]
    auc_roc = sklearn.metrics.roc_auc_score(Y_val, y_pred)

    if auc_roc > best_score:
        print(f'Mejor valor de ROC-AUC encontrado: {auc_roc:.5f}')
        best_score = auc_roc
        best_grid = g
        best_estimator = clf_xgb

end = time.time()
print('ROC-AUC: %0.5f' % best_score)
print('Grilla:', best_grid)
print(f'Tiempo total: {end - start:.2f}s')
print(f'Tiempo/iter: {((end - start) / iterations):.2f}s')


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [22:47:21] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Mejor valor de ROC-AUC encontrado: 0.84389
Mejor valor de ROC-AUC encontrado: 0.84436
Mejor valor de ROC-AUC encontrado: 0.84543
Mejor valor de ROC-AUC encontrado: 0.84605
Mejor valor de ROC-AUC encontrado: 0.84676
ROC-AUC: 0.84676
Grilla: {'colsample_bytree': 0.9994351212622354, 'gamma': 0.53356202855057, 'learning_rate': 0.1953229911665306, 'max_depth': 28, 'min_child_weight': 0.25739375624994676, 'n_estimators': 997, 'reg_lambda': 1.725356240133415, 'subsample': 0.8171756723506819}
Tiempo total: 6157.72s
Tiempo/iter: 61.58s


In [24]:
best_grid = {'colsample_bytree': 0.7626921327598493,
             'gamma': 1.7732342979013198,
             'learning_rate': 0.005523354374740941,
             'max_depth': 34,
             'min_child_weight': 0.469909699204345,
             'n_estimators': 727,
             'reg_lambda': 3.360130676475997,
             'subsample': 0.664076333737366
            }
# Aclaración: acá está "hardcodeado", pero se puede hacer mejor, accediendo a los valores de `best_grid`.

best_estimator = xgb.XGBClassifier(objective = 'binary:logistic',
                                   seed = random_state,
                                   eval_metric = 'auc',
                                   **best_grid)

best_estimator.fit(X_train, Y_train, verbose = True,  eval_set = [(X_val, Y_val)])

# roc_auc_score requiere un array 1D; da lo mismo qué dimensión le pasemos: 90 o 1.
y_pred = best_estimator.predict_proba(X_val)[:, 1]
auc_roc = sklearn.metrics.roc_auc_score(Y_val, y_pred)
print('AUC-ROC validación: %0.5f' % auc_roc)

[0]	validation_0-auc:0.80211
[1]	validation_0-auc:0.81057
[2]	validation_0-auc:0.81315
[3]	validation_0-auc:0.81391
[4]	validation_0-auc:0.82108
[5]	validation_0-auc:0.82327
[6]	validation_0-auc:0.82259
[7]	validation_0-auc:0.82340
[8]	validation_0-auc:0.82187
[9]	validation_0-auc:0.82136
[10]	validation_0-auc:0.82003
[11]	validation_0-auc:0.82068
[12]	validation_0-auc:0.82135
[13]	validation_0-auc:0.82176
[14]	validation_0-auc:0.82203
[15]	validation_0-auc:0.82257
[16]	validation_0-auc:0.82266
[17]	validation_0-auc:0.82324
[18]	validation_0-auc:0.82232
[19]	validation_0-auc:0.82261
[20]	validation_0-auc:0.82289
[21]	validation_0-auc:0.82268
[22]	validation_0-auc:0.82287
[23]	validation_0-auc:0.82286
[24]	validation_0-auc:0.82227
[25]	validation_0-auc:0.82254
[26]	validation_0-auc:0.82191
[27]	validation_0-auc:0.82223
[28]	validation_0-auc:0.82253
[29]	validation_0-auc:0.82243
[30]	validation_0-auc:0.82269
[31]	validation_0-auc:0.82319
[32]	validation_0-auc:0.82333
[33]	validation_0-au

In [25]:
bst = best_estimator.get_booster()
for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
    print('%s: ' % importance_type, bst.get_score(importance_type = importance_type))
    print('--------------')








weight:  {'shuffle': 562.0, 'offline': 7536.0, 'incognito_mode': 474.0, 'obs_id': 116871.0, 'platform_Android OS 11 API 30 (motorola, moto g(9) plus)': 137.0, 'platform_Android OS 11 API 30 (motorola, moto g71 5G)': 99.0, 'platform_Android OS 12 API 31 (motorola, moto g200 5G)': 139.0, 'platform_Android OS 12 API 31 (motorola, moto g71 5G)': 177.0, 'platform_Android OS 12 API 31 (samsung, SM-G990E)': 22.0, 'platform_Android OS 4.1.2 API 16 (samsung, GT-I8260L)': 621.0, 'platform_Android OS 4.1.2 API 16 (samsung, GT-S7390L)': 590.0, 'platform_Android OS 4.4.2 API 19 (samsung, SM-G355M)': 513.0, 'platform_Android OS 4.4.4 API 19 (motorola, XT1045)': 381.0, 'platform_Android OS 5.0.2 API 21 (motorola, MotoE2(4G-LTE))': 656.0, 'platform_Android OS 5.1 API 22 (BLU, BLU LIFE ONE X)': 396.0, 'platform_Android OS 5.1 API 22 (alps, i5)': 286.0, 'platform_Android OS 5.1 API 22 (motorola, XT1032)': 129.0, 'platform_Android OS 5.1 API 22 (motorola, XT1045)': 655.0, 'platform_Android OS 5.1.1 API 2

### **Conjunto de test**

In [28]:
# 1) Saneador de nombres (mismo que usamos antes)
def _sanitize_cols(df):
    return df.rename(columns=lambda c: str(c).replace('[','(').replace(']',')').replace('<','lt'))

X_train = _sanitize_cols(X_train)
X_val   = _sanitize_cols(X_val)
X_test  = _sanitize_cols(X_test)      # <-- faltaba esto

# 2) Alinear columnas y tipos
X_test = X_test.reindex(columns=X_train.columns, fill_value=0).astype('float32')


In [29]:
y_pred = clf_xgb.predict_proba(X_test)[:, 1] 
auc_roc = sklearn.metrics.roc_auc_score(Y_test, y_pred)
print('AUC-ROC test: %0.5f' % auc_roc)


AUC-ROC test: 0.84187


In [31]:
# --- 1) helpers: mismo OHE + "sanitizado" de nombres que usaste en train ---
def _sanitize_cols(df):
    return df.rename(columns=lambda c: str(c).replace('[','(').replace(']',')').replace('<','lt'))

def make_features(df):
    df = df.copy()
    # ¡solo OHE de las categóricas! (no metas la variable a predecir)
    df = pd.get_dummies(
        df,
        columns=['platform', 'conn_country'],
        sparse=True,
        dummy_na=False,
        dtype=int
    )
    df = _sanitize_cols(df)
    return df

# --- 2) construir X_kaggle con las MISMAS columnas y orden que vio el modelo ---
X_kaggle = make_features(test_data)

# si X_train es tu matriz con la que entrenaste:
X_kaggle = X_kaggle.reindex(columns=X_train.columns, fill_value=0)

# --- 3) predecir con el modelo que tengas (best_estimator o clf_xgb) ---
model = globals().get('best_estimator') or globals().get('clf_xgb')
assert model is not None, "No encuentro el modelo entrenado (best_estimator / clf_xgb)."

probas = model.predict_proba(X_kaggle)[:, 1]

# --- 4) armar CSV para Kaggle: columnas obs_id y pred_proba ---
submission = pd.DataFrame({
    'obs_id': test_data['obs_id'].values,
    'pred_proba': probas
})
submission.to_csv('submission_xgb.csv', index=False)
print("Listo: guardé 'submission_xgb.csv'")


Listo: guardé 'submission_xgb.csv'
